In [1]:
import subprocess
from pyspark.sql.types import IntegerType
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import sum



p = subprocess.Popen("hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/aggre| awk '{print $8}' | awk 'NR!=1' ",
shell=True,
stdout=subprocess.PIPE,
stderr=subprocess.STDOUT)
#hdfs://localhost:9000/user/akash/trafficCount/aggre/data1.csv


In [2]:
res = list()
def hdfs_file_read(file):
    rdd = spark.read.csv(file,header=True)
    # rdd=rdd.drop("axlespacings", "axleweights", "numberofaxles","validitycode","duration","temperature","weight","hour","minute","second","millisecond")
    return rdd
for line in p.stdout.readlines():
    file = line.decode().strip()
   
    vehicle_data= hdfs_file_read(file)
    vehicle_data.printSchema()
    vehicle_data= vehicle_data.withColumn("VehicleCount", vehicle_data["VehicleCount"].cast(IntegerType()))
   
    vehicle_data= vehicle_data.withColumn("cosit", vehicle_data["cosit"].cast(IntegerType()))
#cosit =1011 class 1 - 150000
#cosit =1011 class 2 - 150000
#cosit =1011 class 3 - 150000
#cosit =1011 class 4 - 150000
#cosit =1011 class 5 - 150000
#cosit =1011 class 6 - 150000
#cosit =1011 class 7 - 150000
    
    output=vehicle_data.groupBy(vehicle_data.cosit,vehicle_data.day,vehicle_data.month,vehicle_data.year).sum('VehicleCount').sort('sum(VehicleCount)',ascending=False)
  
    output=output.withColumnRenamed("sum(VehicleCount)", "VehicleCount")
#    output.show()
    output.createOrReplaceTempView('packages') 
    query_results = spark.sql('SELECT * FROM packages LIMIT 1') 

#     query_results.show()
#append the results
    res.append(query_results)
    
union_df = reduce(DataFrame.unionAll, res)
# union_df.rdd.map(tuple).saveAsPickleFile("foo")

# union_df.coalesce(1).write.format("csv").save("hdfs://localhost:9000/user/akash/trafficCount/aggre/output")
# union_df.rdd.saveAsTextFile("hdfs://localhost:9000/user/akash/trafficCount/aggre/output")
union_df.show()
    


root
 |-- cosit: string (nullable = true)
 |-- class: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- VehicleCount: string (nullable = true)

root
 |-- cosit: string (nullable = true)
 |-- class: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- VehicleCount: string (nullable = true)

root
 |-- cosit: string (nullable = true)
 |-- class: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- VehicleCount: string (nullable = true)

root
 |-- cosit: string (nullable = true)
 |-- class: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- VehicleCount: string (nullable = true)

root
 |-- cosit: string (nullable = true)
 |-- class: string (nullable =